In [1]:
# Import requested libraries.
# Influxdb > quering InfluxDB database.
# Pandas > loading ".csv" files and handling data.
import os

from influxdb import InfluxDBClient
import pandas as pd

In [2]:
# Define constants for InfluxDB connection.
INFLUXDB_HOST = 'localhost'
INFLUXDB_PORT = 8086
INFLUXDB_DATABASE = 'mydb'
INFLUXDB_USERNAME = 'root'
INFLUXDB_PASSWORD = 'root'
INFLUXDB_SSL = False
INFLUXDB_VERIFY_SSL = False
INFLUXDB_TIMEOUT = 5000

In [3]:
# Create an influx client to interact with database
client = InfluxDBClient(
    host=INFLUXDB_HOST, port=INFLUXDB_PORT,
    username=INFLUXDB_USERNAME, password=INFLUXDB_PASSWORD,
    ssl=INFLUXDB_SSL, verify_ssl=INFLUXDB_VERIFY_SSL,
    timeout=INFLUXDB_TIMEOUT)

In [4]:
# Create new database and connect to it
client.create_database(INFLUXDB_DATABASE)
client.switch_database(INFLUXDB_DATABASE)

In [5]:
# Get list of files to read
DIRPATH = './data/Measurements/'
filenames = os.listdir(DIRPATH)

In [6]:
# Loop over filenames
for filename in filenames:

    # Read csv file
    filepath = DIRPATH + filename
    data = pd.read_csv(filepath, parse_dates=True)

    # Get Timeserie metadata from filename
    filename_without_extension = os.path.splitext(filename)[0]
    location, measurement = filename_without_extension.split('_', 1)
    
    # Prepare and write data to influx database
    serie = [
        {
            'time': row[0],
            'measurement': measurement,
            'fields': {
                'value': row[1],
            },
            'tags': {
                'location': location
            }
        }
        for row in data
    ]
    client.write_points(serie, time_precision='s')

In [7]:
# Request data to check if all OK
result = client.query("SELECT * FROM Temperature LIMIT 5")
some_data = [point for point in result.get_points()]
some_data

[{'location': 'Bathroom', 'time': '2019-06-01T00:00:00Z', 'value': '4'},
 {'location': 'Kitchen', 'time': '2019-06-01T00:00:00Z', 'value': '4'},
 {'location': 'Room1', 'time': '2019-06-01T00:00:00Z', 'value': '4'},
 {'location': 'Room2', 'time': '2019-06-01T00:00:00Z', 'value': '4'},
 {'location': 'Room3', 'time': '2019-06-01T00:00:00Z', 'value': '4'}]